# MVD 11. cvičení


## 1. část - Collaborative Filtering (item-item)

Vytvořte si matici podle příkladu v přednášce na item-item přístup na snímku 24. Využijte centered cosine similarity pro výpočet podobností stejně jako v přednášce a vyberte dva nejbližší sousedy. Ověřte, že je výsledek v přednášce správný a implementujte funkci tak, aby bylo možné jednoduše spočítat i libovolné další vyhodnocení. 

In [21]:
import numpy as np

In [22]:
data_table = [
    [1,    None, 3,    None, None, 5,    None, None, 5,    None, 4,  None],
    [None, None, 5,    4,    None, None, 4,    None, None, 2,    1,  3,  ],
    [2,    4,    None, 1,    2,    None, 3,    None, 4,    3,    5,  None],
    [None, 2,    4,    None, 5,    None, None, 4,    None, None, 2,  None],
    [None, None, 4,    3,    4,    2,    None, None, None, None, 2,  5   ],
    [1,    None, 3,    None, 3,    None, None, 2,    None, None, 4,  None],
]
data_table = np.array(data_table)
data_table[data_table == None] = 0
data_table = data_table.astype(float)
data_table

array([[1., 0., 3., 0., 0., 5., 0., 0., 5., 0., 4., 0.],
       [0., 0., 5., 4., 0., 0., 4., 0., 0., 2., 1., 3.],
       [2., 4., 0., 1., 2., 0., 3., 0., 4., 3., 5., 0.],
       [0., 2., 4., 0., 5., 0., 0., 4., 0., 0., 2., 0.],
       [0., 0., 4., 3., 4., 2., 0., 0., 0., 0., 2., 5.],
       [1., 0., 3., 0., 3., 0., 0., 2., 0., 0., 4., 0.]])

In [23]:
cos_sim = lambda a, b: np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [24]:
def normailize_matrix(matrix: np.ndarray) -> np.ndarray:
    """
    normilize a matrix (mean of each row == 0) of data
    :param data: input data
    :return: centered matrix
    """
    mean_matrix = np.sum(matrix, 1) / np.count_nonzero(matrix, 1)
    result = matrix - mean_matrix.reshape(-1, 1)
    result[np.where(matrix == 0)] = 0
    return result

In [ ]:
def collaborative_filtering(row: int, col: int, matrix: np.ndarray, n_neighboors=2) -> np.float64:
    centered_matrix = normailize_matrix(matrix)

    R, _ = matrix.shape  # get rows number

    # calculate cosinus similarities for all row pairs
    cos_sims = []
    chosen_row = centered_matrix[row]
    for r in range(R):
        cos_sims.append(cos_sim(chosen_row, centered_matrix[r]))

    # set similarities to -inf where in the matrix there is no value in chosen column and similarity row
    similarities = [(idx, sim) for idx, sim in enumerate(cos_sims) if matrix[idx, col] != 0]

    # find n neighboors with the highest similarity
    similarities.sort(key=lambda x: x[1], reverse=True)
    n_sims = similarities[:n_neighboors]
    
    # calculate predicted value based on n-similarities and values from chosen column on n-similarities rows
    indeces, sims = zip(*n_sims)
    sims = np.array(sims)
    indeces = list(indeces)
    predicted_row = np.sum(sims * matrix[indeces, col]) / sims.sum()

    return predicted_row
    


In [28]:
collaborative_filtering(row=0, col=4, matrix=data_table)

np.float64(2.586406866934817)

## Bonus - Content-based Filtering

Stáhněte si Kaggle dataset [Spotify Recommendation system](https://www.kaggle.com/bricevergnou/spotify-recommendation). Z datasetu budete potřebovat:

- data.csv = příznaky k jednotlivým skladbám + příznak liked pro klasifikaci

Úkolem je:

1. Načíst data z csv.
2. Vytvořit train (90 %) a test (10 %) split pomocí knihovny [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html).
3. Vytvořit model logistické regrese (vlastní implementace / využití knihovny sklearn nebo jiné).
4. Vyhodnotit data na testovací sadě (např. metoda score u LogisticRegression).

**Skóre pro uznání úlohy musí být vyšší než 89 %.**

Dobrovolné:
- vytvořit graf predikovaných vs aktuálních hodnot
- využít i jiný model